In [98]:
from bs4 import BeautifulSoup
import requests
import pymongo

In [2]:
url = 'https://www.imdb.com/list/ls052535080/?sort=list_order,asc&st_dt=&mode=detail&page=1'

In [3]:
response = requests.get(url)

In [8]:
soup = BeautifulSoup(response.content, 'lxml')

In [10]:
list_divs = soup.body.find_all('div', {'class':'lister-item'})

In [11]:
len(list_divs)

100

In [18]:
all_list_divs = []
for page in range(1,12):
    url = f'https://www.imdb.com/list/ls052535080/?sort=list_order,asc&st_dt=&mode=detail&page={page}'
    response = requests.get(url)
    print(url, response.status_code)
    if (response.status_code == 200):
        soup = BeautifulSoup(response.content, 'lxml')
        list_divs = soup.body.find_all('div', {'class':'lister-item'})
        all_list_divs += list_divs
        

https://www.imdb.com/list/ls052535080/?sort=list_order,asc&st_dt=&mode=detail&page=1 200
https://www.imdb.com/list/ls052535080/?sort=list_order,asc&st_dt=&mode=detail&page=2 200
https://www.imdb.com/list/ls052535080/?sort=list_order,asc&st_dt=&mode=detail&page=3 200
https://www.imdb.com/list/ls052535080/?sort=list_order,asc&st_dt=&mode=detail&page=4 200
https://www.imdb.com/list/ls052535080/?sort=list_order,asc&st_dt=&mode=detail&page=5 200
https://www.imdb.com/list/ls052535080/?sort=list_order,asc&st_dt=&mode=detail&page=6 200
https://www.imdb.com/list/ls052535080/?sort=list_order,asc&st_dt=&mode=detail&page=7 200
https://www.imdb.com/list/ls052535080/?sort=list_order,asc&st_dt=&mode=detail&page=8 200
https://www.imdb.com/list/ls052535080/?sort=list_order,asc&st_dt=&mode=detail&page=9 200
https://www.imdb.com/list/ls052535080/?sort=list_order,asc&st_dt=&mode=detail&page=10 200
https://www.imdb.com/list/ls052535080/?sort=list_order,asc&st_dt=&mode=detail&page=11 200


In [19]:
len(all_list_divs)

1001

In [136]:
def parse_div(div):
    return {
        'index':int(div.find('h3').find('span', {'class':'lister-item-index'}).text[:-1].replace(',','')),
        'img_url':div.find('img', {'class':'loadlate'})['loadlate'],
        'title' : div.find('h3').find('a').text,
        'title_url' : div.find('h3').find('a')['href'],
        'title_id' : div.find('div', {'class':'lister-item-image ribbonize'})['data-tconst'],
        'year' : div.find('h3').find('span',{'class':'lister-item-year'}).text[1:-1],
        'summary' : div.find_all('p')[1].text.lstrip(),
        'director' : div.find_all('p')[2].find_all('a')[0].text,
        'director_url' : div.find_all('p')[2].find_all('a')[0]['href'],
        'stars' : [x.text for x in div.find_all('p')[2].find_all('a')[1:]],
        'star_urls' : [x['href'] for x in div.find_all('p')[2].find_all('a')[1:]]
    }

In [132]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [133]:
db = client.movies_db

In [137]:
movies = [parse_div(div) for div in all_list_divs]

In [138]:
db.movies.insert_many(movies)

In [139]:
import urllib.request

In [148]:
for movie in movies:
    url = movie['img_url']
    title_id = movie['title_id']
    filename = f'./images/{title_id}.jpg'
    urllib.request.urlretrieve(url,filename)

In [149]:
movies[0]

{'_id': ObjectId('5ae5302b2f16476408e0d0e9'),
 'director': 'Georges Méliès',
 'director_url': '/name/nm0617588/?ref_=ttls_li_dr_0',
 'img_url': 'https://ia.media-imdb.com/images/M/MV5BMzcyYzhlMzctYjg1NS00ODJlLTkxOTUtYmQ4YjgxMWY5MGFlXkEyXkFqcGdeQXVyNDE5MTU2MDE@._V1_UY209_CR2,0,140,209_AL_.jpg',
 'index': 1,
 'star_urls': ['/name/nm0617588/?ref_=ttls_li_st_0',
  '/name/nm0029244/?ref_=ttls_li_st_1',
  '/name/nm0076933/?ref_=ttls_li_st_2',
  '/name/nm1215996/?ref_=ttls_li_st_3'],
 'stars': ['Georges Méliès', 'Victor André', 'Bleuette Bernon', 'Brunnet'],
 'summary': 'A group of astronomers go on an expedition to the Moon.',
 'title': 'A Trip to the Moon',
 'title_id': 'tt0000417',
 'title_url': '/title/tt0000417/?ref_=ttls_li_tt',
 'year': '1902'}